In [13]:
import os 
import numpy as np
import string
import torch
import re
from tqdm import tqdm
from scipy.io import loadmat

## Dataset
The dataset contains the EEG recordings of 15 subjects. For each subject, we have 15 different recordings, each one collected while watching a different movie clip. Each clip is associated to an emotional state amon {sad: -1, neutral: 0, happy: 1}. EEG recordings comprises 62 channels.

N.B. Recordings correspondent to the same movies have the same length, while recordings correspondent to different movies have different length (iun general). How to do? No problem, since we are taking sub windows of the signals.

Data have been preprocessed by downsampling signals to 200Hz, segmentating the signals such that it corresponds to the length of the movie and applying a band-pass filter at 0-75Hz. Since recordings are about 4 minutes long and are now sampled at 200Hz, they contain roughly 48k time points each.

In [2]:
data = loadmat("data/Preprocessed_EEG/1_20131027.mat")

In [5]:
labels = loadmat("data/Preprocessed_EEG/label.mat")
labels["label"].shape

(1, 15)

In [13]:
print(data.keys())
data["djc_eeg3"].shape

dict_keys(['__header__', '__version__', '__globals__', 'djc_eeg1', 'djc_eeg2', 'djc_eeg3', 'djc_eeg4', 'djc_eeg5', 'djc_eeg6', 'djc_eeg7', 'djc_eeg8', 'djc_eeg9', 'djc_eeg10', 'djc_eeg11', 'djc_eeg12', 'djc_eeg13', 'djc_eeg14', 'djc_eeg15'])


(62, 41201)

In [29]:
eeg_files = [fname for fname in os.listdir("data/Preprocessed_EEG") if fname[0] in string.digits]
data_lengths = []
for eeg_file in tqdm(eeg_files):
    raw = loadmat(os.path.join("data/Preprocessed_EEG", eeg_file))
    curr_lengths = []
    pattern = list(raw.keys())[4].split("_")[0]
    for i in range(15):
        curr_lengths.append(raw[f"{pattern}_eeg{i + 1}"].shape[1])
    data_lengths.append(curr_lengths)

data_lengths = np.asarray(data_lengths)
print(data_lengths.shape)
print(np.mean(data_lengths))
print(np.mean(data_lengths, axis=0))
print(np.mean(data_lengths, axis=1))

  0%|          | 0/45 [00:00<?, ?it/s]

100%|██████████| 45/45 [01:03<00:00,  1.40s/it]

(45, 15)
45254.333333333336
[47001. 46601. 41201. 47601. 37001. 39001. 47401. 43201. 53001. 47401.
 47001. 46601. 47001. 47601. 41201.]
[45254.33333333 45254.33333333 45254.33333333 45254.33333333
 45254.33333333 45254.33333333 45254.33333333 45254.33333333
 45254.33333333 45254.33333333 45254.33333333 45254.33333333
 45254.33333333 45254.33333333 45254.33333333 45254.33333333
 45254.33333333 45254.33333333 45254.33333333 45254.33333333
 45254.33333333 45254.33333333 45254.33333333 45254.33333333
 45254.33333333 45254.33333333 45254.33333333 45254.33333333
 45254.33333333 45254.33333333 45254.33333333 45254.33333333
 45254.33333333 45254.33333333 45254.33333333 45254.33333333
 45254.33333333 45254.33333333 45254.33333333 45254.33333333
 45254.33333333 45254.33333333 45254.33333333 45254.33333333
 45254.33333333]


In [31]:
print(np.unique(data_lengths, return_counts=True))
print(np.std(data_lengths, axis=0))

(array([37001, 39001, 41201, 43201, 46601, 47001, 47401, 47601, 53001]), array([ 45,  45,  90,  45,  90, 135,  90,  90,  45]))
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
